<a href="https://colab.research.google.com/github/gabelev/synthetic_vocal_identification/blob/main/find_matches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Authorize me
from google.colab import auth
auth.authenticate_user()

In [ ]:
# %%capture
# Install some dependencies if in colab
# !%apt install ffmpeg
# %pip install spleeter
%pip install tdqm
# %pip install torchaudio
# %pip install librosa
%pip install safetensors
# %pip install pyannote.audio
# %pip install pyannote.core[notebook]
# %pip install tensorflow==2.9.1
# %pip install pyannote.pipeline==1.5.2


  Preparing metadata (setup.py) ... done
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1323 sha256=335558ef614584394978f492e95b2aed4cf54ada256894aa4ea6425f8d4cc59d
  Stored in directory: /root/.cache/pip/wheels/37/31/b8/7b711038035720ba0df14376af06e5e76b9bd61759c861ad92
Successfully built tdqm


In [ ]:
!gsutil -m cp -r gs://singer-id/ai_vocal_id_tensors .

In [ ]:
!gsutil -m cp -r gs://singer-id/vocal_id_tensors .

Copying gs://singer-id/vocal_id_tensors/aerosmith_01-Back_In_The_Saddle_Again.mp3.safetensors...
Copying gs://singer-id/vocal_id_tensors/aerosmith_01-Make_It.mp3.safetensors...
Copying gs://singer-id/vocal_id_tensors/aerosmith_01-Draw_the_Line.mp3.safetensors...
Copying gs://singer-id/vocal_id_tensors/aerosmith_01-Toys_In_The_Attic.mp3.safetensors...
Copying gs://singer-id/vocal_id_tensors/aerosmith_01-Same_Old_Song_and_Dance.mp3.safetensors...
Copying gs://singer-id/vocal_id_tensors/aerosmith_02-I_Wanna_Know_Why.mp3.safetensors...
Copying gs://singer-id/vocal_id_tensors/aerosmith_02-F_I_N_E_.mp3.safetensors...
Copying gs://singer-id/vocal_id_tensors/aerosmith_01-Young_Lust.mp3.safetensors...
Copying gs://singer-id/vocal_id_tensors/aerosmith_02-Uncle_Salty.mp3.safetensors...
Copying gs://singer-id/vocal_id_tensors/aerosmith_02-Last_Child.mp3.safetensors...
Copying gs://singer-id/vocal_id_tensors/aerosmith_02-Somebody.mp3.safetensors...
Copying gs://singer-id/vocal_id_tensors/aerosmith_

In [ ]:
# for testing
#!gsutil -m cp gs://singer-id/vocal_id_tensors/beatles_08-Happiness_Is_A_Warm_Gun.mp3.safetensors .
#!gsutil -m cp gs://singer-id/ai_vocal_id_tensors/0xoPoN-Etgg.safetensors .
# !gsutil -m cp gs://singer-id/ai_vocal_id_tensors/C0Mja5SdZEk.safetensors .

Copying gs://singer-id/ai_vocal_id_tensors/C0Mja5SdZEk.safetensors...
/ [1/1 files][  2.1 KiB/  2.1 KiB] 100% Done                                    
Operation completed over 1 objects/2.1 KiB.                                      


In [ ]:
!mkdir humans/

In [ ]:
!mkdir ais/

In [ ]:
from safetensors import safe_open

def get_safetensor(path):
    with safe_open(path, framework="pt", device="cpu") as f:
        X = f.get_tensor("tensor")
    return X

In [ ]:
import os
import tqdm

input_path = "/content/vocal_id_tensors/"

# create dict of tensors
humans = {}

for filename in os.listdir(input_path):
    humans[filename.split(".")[0]] = get_safetensor(input_path + filename)


In [ ]:
import os
import tqdm

input_path = "/content/ai_vocal_id_tensors/"

# create dict of tensors
ais = {}

for filename in os.listdir(input_path):
    ais[filename.split(".")[0]] = get_safetensor(input_path + filename)


In [ ]:
import numpy as np

from scipy.spatial.distance import cdist

def find_distance(e1, e2):
    return cdist(
        np.reshape(e1.detach().cpu().numpy(), (1, -1)),
        np.reshape(e2.detach().cpu().numpy(), (1, -1)),
        metric="cosine")[0,0]
#

In [ ]:
import os
import tqdm


def match(ais, humans):
# create dict of tensors
    matches = {}

    for ai_key, ai_value in ais.items():
        min_score = 1000.00
        for key, value in humans.items():
            new_score = find_distance(ai_value, value)
            if new_score <= min_score:
                result = {}
                result[key] = new_score
                matches[ai_key] = result
                min_score = new_score
    return matches



In [ ]:
matches = match(ais, humans)

In [ ]:
matches

{'GjsSzqAWFAU': {'garth_brooks_07-The_Beaches_of_Cheyenne': 0.22276650791432273},
 'Lr8HJ3oW9-8': {'madonna_05-Cry_Baby': 0.3388562325244654},
 '-Vo2vDsFdfw': {'dave_matthews_band_09-Digging_a_Ditch': 0.32402270593892757},
 'dcGKHNDRHkw': {'tori_amos_06-i_i_e_e_e': 0.15025662521189587},
 'YNy8VqlPX6Q': {'led_zeppelin_06-Living_Loving_Maid_She_s_Just': 0.30638113665321565},
 'uQnn0aRTnJc': {'tori_amos_08-She_s_Your_Cocaine': 0.3644727073205224},
 'SDR9GuT_86w': {'beatles_11-Every_Little_Thing': 0.2090460066860007},
 '5IYpleg9hJk': {'creedence_clearwater_revival_04-Good_Golly_Miss_Molly': 0.2770931556312537},
 'C0Mja5SdZEk': {'madonna_11-Act_Of_Contrition': 0.4061475191603545},
 '7QdxDJxJuAc': {'dave_matthews_band_10-Big_Eyed_Fish': 0.2024213765425742},
 'cOBSJKRrMYw': {'led_zeppelin_06-Living_Loving_Maid_She_s_Just': 0.19923715095680028},
 'U-y6K8X3hYM': {'beatles_10-For_No_One': 0.18552682021030598},
 '0xoPoN-Etgg': {'tori_amos_13-Ireland': 0.34869394948512256},
 'PWQG3mwH8i8': {'led_z